In [1]:
from dataclasses import asdict
import pandas as pd
from pprint import pprint
import networkx as nx
import json

from martignac.nomad.entries import (
    get_all_my_uploads,
    get_entries_in_database,
    DEFAULT_DATABASE,
)
from martignac.nomad.uploads import delete_upload

from martignac.workflows.solute_generation import project as solute_gen_project
from martignac.workflows.solvent_generation import project as solvent_gen_project
from martignac.workflows.solute_solvation import project as solute_solvation_project
from martignac.workflows.alchemical_transformation import project as alchem_transf_project

from martignac.nomad.workflows import NomadWorkflow

24-03-27 16:34:35 - MDAnalysis.coordinates.AMBER - WARNING - netCDF4 is not available. Writing AMBER ncdf files will be slow.
24-03-27 16:34:36 - pymbar.timeseries - WARNING - Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
24-03-27 16:34:36 - pymbar.mbar_solvers - WARNING - 
********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************
24-03-27 16:34:36 - pymbar.mbar_solvers - INFO - JAX was either not detected or disabled, using standard NumPy and SciPy


In [2]:
def clear_jobs_for_project(project):
    for job in list(project.find_jobs({})):
        job.clear()

## Nomad access

In [10]:
my_uploads = get_all_my_uploads()
df = pd.json_normalize(asdict(u) for u in my_uploads)
df = df[df.published == False]
df

,upload_id,upload_create_time,process_running,current_process,process_status,last_status_message,errors,warnings,coauthors,coauthor_groups,...,main_author.first_name,main_author.last_name,main_author.username,main_author.affiliation,main_author.affiliation_address,main_author.email,main_author.is_oasis_admin,main_author.is_admin,main_author.repo_user_id,main_author.created


In [7]:
# delete_upload(df.iloc[1].upload_id, False)

In [38]:
def remove_unpublished_entries():
    my_uploads = get_all_my_uploads()
    df = pd.json_normalize(asdict(u) for u in my_uploads)
    df = df[df.published == False]
    for entry in df.values:
        delete_upload(entry.upload_id, use_prod=entry.use_prod)

In [39]:
remove_unpublished_entries()

## Solute generation

In [3]:
clear_jobs_for_project(solute_gen_project)

In [4]:
solute_gen_project

,sp.type,sp.solute_name
bc3c7ca12d4e7b0fa984e8cbb813acbf,solute,P6


In [6]:
solute_gen_project.run()

24-03-26 15:49:59 - flow.project - INFO - Executing 1 operation(s) (Pass #01)...
24-03-26 15:49:59 - flow.project - INFO - Execute operation 'generate_nomad_workflow(bc3c7ca12d4e7b0fa984e8cbb813acbf)'...
24-03-26 15:49:59 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/martignac/workspaces/solute_generation/workspace/bc3c7ca12d4e7b0fa984e8cbb813acbf'.
24-03-26 15:49:59 - martignac.nomad.workflows - INFO - building workflow graph for solute_generation.archive.yaml
24-03-26 15:49:59 - signac.job - INFO - Leave workspace.
24-03-26 15:49:59 - flow.project - INFO - Operation 'generate_nomad_workflow(bc3c7ca12d4e7b0fa984e8cbb813acbf)' exceeds max. # of allowed passes (1).


In [7]:
workflow = NomadWorkflow(solute_gen_project, list(solute_gen_project.find_jobs())[0])
workflow

NomadWorkflow(project=SoluteGenFlow('/Users/bereau/work/projects/martignac/workspaces/solute_generation'), job=Job(project=SoluteGenFlow('/Users/bereau/work/projects/martignac/workspaces/solute_generation'), statepoint={'type': 'solute', 'solute_name': 'P6'}))

## Solvent generation

In [7]:
solvent_gen_project

,sp.type,sp.solvent_name
3af62c304ca8f079d8a551f0710b7f2a,solvent,OCO
598503070d283f2bd13d71a109ba67ac,solvent,HD


In [3]:
solvent_gen_project.run()

In [6]:
clear_jobs_for_project(solvent_gen_project)

## Solute solvation

In [11]:
clear_jobs_for_project(solute_solvation_project)

In [12]:
solute_solvation_project

,sp.type,sp.solvent_name,sp.solute_name
d220d93969fe6913f625e887742cfcee,solute_solvation,HD,P6


In [14]:
list(solute_solvation_project.find_jobs())[0].clear()

In [13]:
solute_solvation_project.run()

24-03-26 11:47:32 - flow.project - INFO - Executing 2 operation(s) (Pass #01)...
24-03-26 11:47:32 - flow.project - INFO - Execute operation 'symlink_itp_and_mdp_files(d220d93969fe6913f625e887742cfcee)'...
24-03-26 11:47:32 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/martignac/workspaces/solute_solvation/workspace/d220d93969fe6913f625e887742cfcee'.
24-03-26 11:47:32 - martignac.utils.martini_flow_projects - INFO - generating symbolic links for d220d93969fe6913f625e887742cfcee @ SoluteSolvationFlow
24-03-26 11:47:32 - signac.job - INFO - Leave workspace.
24-03-26 11:47:32 - flow.project - INFO - Execute operation 'symlink_itp_and_mdp_files(54276bca258fecb21fbe62c7dbc7a2ef)'...
24-03-26 11:47:32 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/martignac/workspaces/solute_solvation/workspace/54276bca258fecb21fbe62c7dbc7a2ef'.
24-03-26 11:47:32 - martignac.utils.martini_flow_projects - INFO - generating symbolic links for 54276bca258fecb21fbe62c7dbc

Setting the LD random seed to -101462034

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 3 bonded neighbours molecule type 'P6'
Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 0 Mb of data


Number of degrees of freedom in T-Coupling group System is 0.00

GROMACS reminds you: "It seems likely that significant software contributions to existing scientific software projects are not likely to be rewarded through the traditional reputation economy of science. Together these factors provide a reason to expect the over-production of independent scientific software packages, and the underproduction of collaborative projects in which later academics build on the work of earlier ones." (Howison & Herbsleb)

                :-) GROMACS - gmx mdrun, 2023.1-conda_forge (-:

Executable:   /Users/bereau/soft/conda/bin.SSE2/gmx
Data prefix:  /Users/bereau/soft/conda
Working dir:  /Users/bereau/work/projects/martignac/workspaces/solute_generation/workspace/bc3c7ca12d4e7b0fa984e8cbb813acbf
Command line:
  gmx mdrun -nt 1 -deffnm solute_equilibrate -v

Compiled SIMD: SSE2, but for this host/run SSE4.1 might be better (see log).
Reading file solute_equilibrate.tpr, VERSION 2023.1-conda_forge

Setting the LD random seed to -303219

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 3 bonded neighbours molecule type 'P6'

Setting gen_seed to -1195377002

Velocities were taken from a Maxwell distribution at 300 K
Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.100 nm, buffer size 0.000 nm

Set rlist, assuming 4x4 atom pair-list, to 1.100 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 0 Mb of data


24-03-26 11:47:35 - martignac.utils.martini_flow_projects - INFO - made connection to dataset HJdEI1q4SV-c5Di43BTT_Q
24-03-26 11:47:35 - martignac.utils.misc - INFO - Zipping /Users/bereau/work/projects/martignac/workspaces/solute_generation/workspace/bc3c7ca12d4e7b0fa984e8cbb813acbf to /Users/bereau/work/projects/martignac/workspaces/solute_generation/workspace/bc3c7ca12d4e7b0fa984e8cbb813acbf.zip
24-03-26 11:47:35 - martignac.utils.misc - INFO - Moved archive to /Users/bereau/work/projects/martignac/workspaces/solute_generation/workspace/bc3c7ca12d4e7b0fa984e8cbb813acbf/bc3c7ca12d4e7b0fa984e8cbb813acbf.zip
24-03-26 11:47:35 - martignac.nomad.uploads - INFO - uploading file /Users/bereau/work/projects/martignac/workspaces/solute_generation/workspace/bc3c7ca12d4e7b0fa984e8cbb813acbf/bc3c7ca12d4e7b0fa984e8cbb813acbf.zip on test server
24-03-26 11:47:35 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads
24-03-26 11:47:35 - martignac.n

...

Step=  115, Dmax= 1.9e-02 nm, Epot= -1.79366e+04 Fmax= 1.15946e+02, atom= 1404
Step=  116, Dmax= 2.3e-02 nm, Epot= -1.79395e+04 Fmax= 2.55461e+02, atom= 1404
Step=  117, Dmax= 2.8e-02 nm, Epot= -1.79453e+04 Fmax= 1.83776e+02, atom= 1404
Step=  119, Dmax= 1.7e-02 nm, Epot= -1.79503e+04 Fmax= 8.25816e+01, atom= 1404
Step=  120, Dmax= 2.0e-02 nm, Epot= -1.79548e+04 Fmax= 2.28886e+02, atom= 1404
Step=  121, Dmax= 2.4e-02 nm, Epot= -1.79603e+04 Fmax= 1.53508e+02, atom= 1404
Step=  122, Dmax= 2.9e-02 nm, Epot= -1.79614e+04 Fmax= 2.99681e+02, atom= 1404
Step=  123, Dmax= 3.4e-02 nm, Epot= -1.79672e+04 Fmax= 2.49373e+02, atom= 1404
Step=  125, Dmax= 2.1e-02 nm, Epot= -1.79738e+04 Fmax= 7.78714e+01, atom= 1404
Step=  126, Dmax= 2.5e-02 nm, Epot= -1.79766e+04 Fmax= 3.23165e+02, atom= 1404
Step=  127, Dmax= 3.0e-02 nm, Epot= -1.79853e+04 Fmax= 1.50591e+02, atom= 1404
Step=  129, Dmax= 1.8e-02 nm, Epot= -1.79893e+04 Fmax= 1.35899e+02, atom= 1404
Step=  130, Dmax= 2.1e-02 nm, Epot= -1.79924e+04 Fma

Setting the LD random seed to -541635077

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 3 bonded neighbours molecule type 'P6'

Excluding 1 bonded neighbours molecule type 'HD'
Analysing residue names:
There are:   391      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.287 nm, buffer size 0.187 nm

Set rlist, assuming 4x4 atom pair-list, to 1.201 nm, buffer size 0.101 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 0 Mb of data


step 49900, remaining wall clock time:     0 s          
Writing final coordinates.
step 50000, remaining wall clock time:     0 s          
NOTE: 12 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:       13.939       13.940      100.0
                 (ns/day)    (hour/ns)
Performance:     6198.328        0.004

GROMACS reminds you: "Just Because the Sun Wants a Place In the Sky" (F. Zappa)

24-03-26 11:48:19 - martignac.utils.martini_flow_projects - INFO - logging log file for gromacs simulation equilibrate @ d220d93969fe6913f625e887742cfcee
24-03-26 11:48:19 - flow.project - INFO - Execute operation 'equilibrate(54276bca258fecb21fbe62c7dbc7a2ef)'...
24-03-26 11:48:19 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/martignac/workspaces/solute_solvation/workspace/54276bca258fecb21fbe62c7dbc7a2ef'.
24-03-26 11:48:19 - signac.job - INFO 

Setting the LD random seed to 1859882623

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 3 bonded neighbours molecule type 'P6'

Excluding 1 bonded neighbours molecule type 'OCO'
Analysing residue names:
There are:   520      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.292 nm, buffer size 0.192 nm

Set rlist, assuming 4x4 atom pair-list, to 1.180 nm, buffer size 0.080 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 0 Mb of data


step 49900, remaining wall clock time:     0 s          
Writing final coordinates.
step 50000, remaining wall clock time:     0 s          
NOTE: 11 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:       15.153       15.154      100.0
                 (ns/day)    (hour/ns)
Performance:     5701.732        0.004

GROMACS reminds you: "The determined Real Programmer can write FORTRAN programs in any language." (Ed Post)

24-03-26 11:48:35 - martignac.utils.martini_flow_projects - INFO - logging log file for gromacs simulation equilibrate @ 54276bca258fecb21fbe62c7dbc7a2ef
24-03-26 11:48:35 - flow.project - INFO - Executing 2 operation(s) (Pass #07)...
24-03-26 11:48:35 - flow.project - INFO - Execute operation 'generate_nomad_workflow(d220d93969fe6913f625e887742cfcee)'...
24-03-26 11:48:35 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/

## Alchemical transformations

In [2]:
alchem_transf_project

,sp.type,sp.solvent_name,sp.solute_name,sp.lambda_state,doc.AlchemicalTransformationFlow,doc.SoluteGenFlow,doc.SolventGenFlow,doc.SoluteSolvationFlow
2ba90a7767b992613270a32ede3ee231,alchemical_transformation,OCO,P6,0,"{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru..."
5ba01cdf44bf4acf957978d4f923cfb5,alchemical_transformation,OCO,P6,1,"{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru..."
3cac983addf8b875eeb3b6488b4983d0,alchemical_transformation,OCO,P6,2,"{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru..."
4128aa1e7e63365ae3a68f4f95b1506c,alchemical_transformation,OCO,P6,3,"{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru...","{'files_symlinked': True, 'fetched_nomad': Tru..."


In [5]:
clear_jobs_for_project(alchem_transf_project)

In [6]:
alchem_transf_project.doc.clear()

In [8]:
alchem_transf_project.run()

24-03-27 16:04:38 - flow.project - INFO - Executing 1 operation(s) (Pass #01)...
24-03-27 16:04:38 - flow.project - INFO - Execute operation 'generate_nomad_workflow(agg-c7e897f059b80c0e2f960b2c15647e98)'...
24-03-27 16:04:38 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/martignac/workspaces/alchemical_transformation/workspace/2ba90a7767b992613270a32ede3ee231'.
24-03-27 16:04:38 - signac.job - INFO - Leave workspace.
24-03-27 16:04:38 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/martignac/workspaces/alchemical_transformation/workspace/5ba01cdf44bf4acf957978d4f923cfb5'.
24-03-27 16:04:38 - signac.job - INFO - Leave workspace.
24-03-27 16:04:38 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/martignac/workspaces/alchemical_transformation/workspace/3cac983addf8b875eeb3b6488b4983d0'.
24-03-27 16:04:38 - signac.job - INFO - Leave workspace.
24-03-27 16:04:38 - signac.job - INFO - Enter workspace '/Users/bereau/work/projects/marti

{'workflow2': {'inputs': [{'name': 'production',
                           'section': '../upload/archive/mainfile/alchemical_transformation_production-0.log#/run/0/calculation/-1'},
                          {'name': 'compute_free_energy',
                           'section': '#/run/0/calculation/-1'}],
               'outputs': [{'name': 'production',
                            'section': '../upload/archive/mainfile/alchemical_transformation_production-0.log#/run/0/calculation/-1'},
                           {'name': 'compute_free_energy',
                            'section': '#/run/0/calculation/-1'}],
               'tasks': [{'inputs': [{'name': 'input',
                                      'section': '../upload/archive/mainfile/alchemical_transformation_production-0.log#/run/0/calculation/-1'}],
                          'm_def': 'nomad.datamodel.metainfo.workflow.TaskReference',
                          'name': 'production',
                          'outputs': [{'name': 